In [1]:
# Importing Necessary Libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

%matplotlib inline

import time
from sklearn.ensemble import IsolationForest
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.seasonal import seasonal_decompose


import warnings
warnings.filterwarnings("ignore")

In [2]:
o_data = pd.read_csv('HomeC14.csv')
data = o_data.copy()

In [3]:
data.columns = [col.replace(' [kW]', '') for col in data.columns]
data['Furnace'] = data[['Furnace 1', 'Furnace 2']].sum(axis=1)
data['Kitchen'] = data[['Kitchen', 'Kitchenette', ]].sum(axis=1)
data.drop(['Furnace 1', 'Furnace 2', 'Kitchenette'], axis=1, inplace=True)
data['icon'].nunique()
data['summary'].unique()
data.isna().sum()
data[data.isnull().any(axis=1)]
data = data.drop(index=503910,axis=0)
data.cloudCover.dtype
data['cloudCover'].unique()
data['cloudCover'].replace('cloudCover', method='bfill', inplace=True)
data['cloudCover'] = data['cloudCover'].astype('float')
data['time'] = pd.to_datetime(data['time'], unit='s')
data['time'] = pd.DatetimeIndex(pd.date_range('2016-01-01 05:00', periods=len(data),  freq='min'))
data['year'] = data['time'].apply(lambda x:x.year)
data['month'] = data['time'].apply(lambda x:x.month)
data['day'] = data['time'].apply(lambda x:x.day)
data['hour'] = data['time'].apply(lambda x:x.hour)
data['minute'] = data['time'].apply(lambda x:x.minute)

In [4]:
def hours_to_timing(hour):
    if hour in [22, 23, 0, 1, 2, 3]:
        timing = 'Night'
    elif hour in range(4, 12):
        timing = 'Morning'
    elif hour in range(12, 17):
        timing = 'Afternoon'
    elif hour in range(17, 22):
        timing = 'Evening'
    else:
        timing = 'X'
    return timing

In [5]:
data['timing'] = data['hour'].apply(hours_to_timing)

In [6]:
def Season(month):
    if month in [1,2,3]:
        season = 'Winter'
    elif month in [4,5,6]:
        season = 'Spring'
    elif month in [7,8,9]:
        season = 'Summer'
    elif month in [10,11,12]:
        season = 'Autumn'            
    else:
        season='X'
            
    return season

In [7]:
data['season'] = data['month'].apply(Season)

In [9]:
new_csv_file = 'EXTRACT.csv'
data.to_csv(new_csv_file, index=False)

In [8]:
q=np.where(data['use'] == data['House overall'])
r=np.where(data['gen'] == data['Solar'])
data = data.drop(['gen','use'],axis=1)
categorical_features=[features for features in data.columns if data[features].dtypes=="O" and features not in ['timing']]
numerical=[features for features in data.columns if data[features].dtypes!="O" and features not in ['time','year', 'month','day', 'hour', 'minute']]
object_nan = [feature for feature in data.columns if data[feature].isnull().sum()>1 and data[feature].dtypes=='O']
numerical_nan = [feature for feature in data.columns if data[feature].isnull().sum()>1 and data[feature].dtypes!='O']
energy_data = data[['House overall', 'Dishwasher', 'Home office',
       'Fridge',  'Garage door',  'Well', 'Microwave',
       'Living room', 'Furnace', 'Kitchen']]
weather_data = data[['Solar','temperature', 'humidity', 'visibility', 'apparentTemperature',
       'pressure', 'windSpeed', 'cloudCover', 'windBearing', 'precipIntensity',
       'dewPoint', 'precipProbability']]
energy_data.index = pd.to_datetime(energy_data.index)
weather_data.index = pd.to_datetime(weather_data.index)
energy_data = energy_data.set_index(data['time'])
weather_data = weather_data.set_index(data['time'])
energy_per_day = energy_data.resample('D').sum()
energy_per_week = energy_data.resample('W').sum()
energy_per_month = energy_data.resample('M').sum()
energy_per_hour = energy_data.resample('H').sum()
weather_per_day = weather_data.resample('D').mean()
weather_per_month = weather_data.resample('M').mean()
weather_per_week = weather_data.resample('W').mean()
weather_per_hour = weather_data.resample('H').mean()